In [1]:
import os
import pandas as pd

In [2]:
base_dir = '/home/jovyan/full_runs/'

In [3]:
sub_dirs = os.listdir(base_dir)

In [16]:
cfgs = []
for sub_dir in sub_dirs:
    if len(sub_dir) != 18 or sub_dir.startswith('.'):
        continue
    cfg_path = os.path.join(base_dir, sub_dir, 'config.csv')
    if not os.path.exists(cfg_path):
        continue
    cfg = pd.read_csv(cfg_path, index_col=0).squeeze()
    cfg['completed'] = '0'
    cfg['exp_path'] = os.path.join(base_dir, sub_dir) 
    key = 'eval' if cfg['final_eval'] == '1' else 'valid'
    history_path = os.path.join(base_dir, sub_dir, 'history.csv')
    if os.path.exists(history_path):
        history = pd.read_csv(history_path, index_col=0)
        if str(len(history)) == cfg['n_epochs']:
            cfg['completed'] = '1'
            cfg[f'train_end_{key}_trial_age_mae'] = history.tail(1)[f'{key}_age_mae'].squeeze()
            cfg[f'valid_best_{key}_trial_age_mae'] = history[history[f'{key}_age_mae_best']].tail(1)[f'{key}_age_mae'].squeeze()
            #cfg[f'train_end_{key}_loss'] = history.tail(1)[f'{key}_loss'].squeeze()
            #cfg[f'valid_best_{key}_loss'] = history[history[f'{key}_loss_best']].tail(1)[f'{key}_loss'].squeeze()
    cfgs.append(cfg)
cfgs = pd.concat(cfgs, axis=1, ignore_index=True).sort_index()

In [17]:
def prune(cfgs):
    cfgs = cfgs.T
    # show only completed valid runs
    cfgs = cfgs[(cfgs.completed == '1') & (cfgs.final_eval == '0') & (cfgs.n_epochs == '35') & (cfgs.subset == 'normal')]
    # only show those columns where not all values are equal
    cfgs = cfgs[cfgs.columns[[cfgs[c].nunique() > 1 for c in cfgs.columns]]]
    return cfgs

In [18]:
cfgs = prune(cfgs)

In [19]:
cfgs.sort_values('exp_path', ascending=False)

,exp_path,loss,train_end_valid_trial_age_mae,valid_best_valid_trial_age_mae
22,/home/jovyan/full_runs/220809091002554818,mse,7.139167,7.120466
13,/home/jovyan/full_runs/220808145130107294,huber,7.373691,7.318414
23,/home/jovyan/full_runs/220808145129781057,mae,7.25364,7.189566
24,/home/jovyan/full_runs/220730071804431899,mae,7.136507,7.115298
12,/home/jovyan/full_runs/220730071802841704,mse,7.207405,7.183028
4,/home/jovyan/full_runs/220729164826192986,huber,7.138583,7.088176
28,/home/jovyan/full_runs/220726013704644042,mse,7.165513,7.126703
19,/home/jovyan/full_runs/220726012902832235,mae,7.180033,7.150732
17,/home/jovyan/full_runs/220725141944002603,huber,7.029969,6.998621
29,/home/jovyan/full_runs/220725094516530336,mae,7.138562,7.087473


In [44]:
import shutil